# Montu Python 
## Astronomical ephemerides for the Ancient World
## Test: Heka (/ḥkꜣ/)


I would prefere to use Ununti (/wnwntj/) that means Astronomer.

In [4]:
# Montu packages and data
from montu import *

# These magic commands are intended to allow updating the packages if they change
%load_ext autoreload
%autoreload 2

In [5]:
class Seba(object):
    """This is the general class for all celestial objects in MontuPython

    The name of the class come from the word for star in acnient egyptina, seba /sbꜣ/.
    """
    def __init__(self):
        pass

In [10]:
class Heka(object):
    def move_to(sebau,mtime):
        pass

    #====================================
    # Public methods
    #====================================
    def apply_precession(sebau,mtime,inplace=False,bar=False):
        """Precess coordinates of a celestial object (seba) of a list of them (sebau).
        It can also be a list of coordinates of the same object.

        Parameters:
            sebau: Series | DataFrame
                Data containing the information on sebau.
                The object must have the following columns or attributes:
                    DecJ2000: Declination in J2000 [deg]
                    RAJ2000: Right ascension in J2000 [hours]

        Return:
            If inplace is False:
                A sebau object with the same attributes that the input one 
                but with additional attributes and columns:
                    Epoch: 
                    RAEpoch:
                    DecEpoch:
            Else:
                The sebau is updated including new columns.

        Comments:
            If the object is a star, proper motion is applied. If it is a planetary object, 
            no proper mootion is applied.
        """
        sebau_is_series = False
        if len(sebau.shape) == 1:
            sebau_is_series = True
            series = sebau
            sebau = pd.DataFrame([sebau])
        elif not inplace:
            sebau = copy.deepcopy(sebau)

        if bar:
            bar = tqdm.tqdm
        else:
            bar = lambda x:x

        for index in bar(sebau.index):
            seba = sebau.loc[index]

            # Get cordinates
            DecJ2000 = seba.DecJ2000
            RAJ2000 = seba.RAJ2000

            # Normal vector pointing to J2000 coordinates
            uJ2000 = spy.latrec(1,15*RAJ2000*DEG,DecJ2000*DEG)
            
            # Transform vector
            uEpoch = spy.mxv(mtime.M_J2000_Epoch,uJ2000)

            # Get spherical coordinates of new vector
            r,RA,Dec = spy.recrad(uEpoch)
            RAEpoch = RA*RAD/15
            DecEpoch = Dec*RAD

            # Get result
            results = dict(
                RAEpoch = RAEpoch,
                DecEpoch = DecEpoch,
                # Deprecated: remove when debuuging is done
                RAEpoch_comp = RAEpoch,
                DecEpoch_comp = DecEpoch,
            )

            if not sebau_is_series:
                # Add fields to object
                sebau.loc[index,results.keys()] = results.values()

        # Add epoch to object
        sebau.epoch = mtime.jed
                
        if not inplace:
            return sebau
        elif sebau_is_series:
            for key,item in results.items():
                series[key] = item

    def where_in_sky(sebau,site,mtime,
                     inplace=False,bar=False,
                     fields=Dictobj(Dec='DecEpoch',RA='RAEpoch')):
        """Transform from equatorial at Epoch to Alt Azimutal
        """
        # Update site
        site.update_site(mtime)

        sebau_is_series = False
        if len(sebau.shape) == 1:
            sebau_is_series = True
            series = sebau
            sebau = pd.DataFrame([sebau])
        elif not inplace:
            sebau = copy.deepcopy(sebau)

        if bar:
            bar = tqdm.tqdm
        else:
            bar = lambda x:x

        for index in bar(sebau.index):
            seba = sebau.loc[index]

            # Get coordinates
            Dec = seba[fields.Dec]
            RA = seba[fields.RA]

            # Compute hour angle
            HA = np.mod(site.ltst - RA,24)

            # Elevation and azimuth
            az,el = Heka.__to_altaz(HA,Dec,site.lat)

            # Atmospheric refraction

            # Compute airmass 
            
            # Results
            results = dict(
                HA = HA,
                az = az,
                el = el,
                # Deprecated: remove when debuuging is done
                HA_comp = HA,
                az_comp = az,
                el_comp = el,
            )

            if not sebau_is_series:
                # Add fields to seba
                sebau.loc[index,results.keys()] = results.values()
                
        if not inplace:
            return sebau
        elif sebau_is_series:
            for key,item in results.items():
                series[key] = item
        pass

    def path_in_sky(seba,site,mtime,duration=24*HOUR,deltat=1*HOUR,tai2sid=TAI_TO_SID):
        """Propagate local position of a body at a given sky coordinates

        Parameters:
            seba: Object information.
                Pandas series containing the coordinates of the object.
                The series should have the attributes: RAEpoch, DecEpoch, pm_ra and pm_dec.

            site: ObservingSite:
                Observing site.

            duration: float [seconds], default = 24*HOUR:
                Duration of the motion.

            deltat: float [seconds], default = 1*HOUR
                Step size.

            tai2sid: float, default = TAI_TO_SID:
                Ratio of sidereal day to TAI day.
                It can be computed using MonTime.tai_to_sid(mtime)
        
        Return:

            daily_df: DataFrame:
                Containing the following columns:
                    tt: Terrestrial time.
                    HA: Hour angle.
                    az, el: Azimuth and elevation
                    isvisible: True if object is visible.
        """
        
        # Update site
        site.update_site(mtime)

        # Initial time
        tt0 = mtime.tt

        # Initial hour angle
        # Get coordinates
        Dec0 = float(seba.DecEpoch)
        RA0 = float(seba.RAEpoch)
        HA0 = np.mod(site.ltst - RA0,24)

        results = []
        for i,tt in enumerate(Montu.arange(tt0,tt0+duration,deltat)):

            HA,az,el = Heka.__move_in_path(HA0,Dec0,site.lat,tt-tt0,tai2sid=tai2sid)
            # Results
            results += [dict(
                tt = tt,
                HA = HA,
                az = az,
                el = el,
                isvisible = True if el>0 else False,
            )]
            
        path = pd.DataFrame(results)
        return path

    def is_reachable(sebau,mtime):
        pass
        
    def is_visible(sebau,mtime):
        pass
    
    def when_rise_culminates_set(sebau,mtime):
        pass
    
    def when_appear_disappear(sebau,mtime):
        pass

    def next_solar_stations(mtime):
        """Calculate next equinoxes and solstices.
        """
        pass

    def previous_solar_stations(mtime):
        """Calculate previous equinoxes and solstices.
        """
        pass

    def next_lunar_quarters(mtime):
        pass

    def previous_lunar_quarters(mtime):
        pass

    #====================================
    # Internal methods
    #====================================
    def __to_altaz(HA,Dec,lat):
        """Transform from local equatorial to azimuth and elevation
        """
        # Elevation and azimuth
        el = np.arcsin(np.sin(Dec*DEG)*np.sin(lat*DEG) + \
                    np.cos(Dec*DEG)*np.cos(lat*DEG)*np.cos(HA*15*DEG))*RAD
        az = np.arctan2(-np.sin(HA*15*DEG)*np.cos(Dec*DEG)/np.cos(el*DEG),
                        (np.sin(Dec*DEG) - np.sin(lat*DEG)*np.sin(el*DEG))/\
                            (np.cos(lat*DEG)*np.cos(el*DEG)))*RAD
        az = np.mod(az,360)
        return az,el
    
    def __move_in_path(HA0,Dec0,lat,deltat,tai2sid=TAI_TO_SID,pm_ra=0,pm_dec=0):
        """Propagate local position of a body at a given sky coordinates

        Parameters:
            HA0 [hours], Dec0 [deg]:
                Initial local sky coordinates of the object.

            lat: float [deg]:
                Latitude of the observing site

            pm_ra, pm_dec: float [uarcsec], default = 0:
                Propet motion in RA and in Dec.

            tai2sid: float, default = TAI_TO_SID:
                Ratio of sidereal day to TAI day.
                It can be computed using MonTime.tai_to_sid(mtime).

        Return:
            HA: float [hour]:
                Hour angle.
            az, el: float [degree]:
                Azimuth and elevation            
        """
        # Equatorial coordinates
        Dec = Dec0 + pm_dec*UARCSEC*deltat
        HA = HA0 + pm_ra*UARCSEC*deltat/15
        # Advance in sidereal time
        dst = tai2sid*deltat/HOUR # Advanced hours
        # Hour angle after deltat
        HA = np.mod(HA + dst,24)
        # Elevation and azimuth
        az,el = Heka._to_altaz(HA,Dec,lat)
        return HA,az,el

    #====================================
    # Static methods
    #====================================


In [12]:
Montu.get_methods(Heka)

['apply_precession',
 'is_reachable',
 'is_visible',
 'move_to',
 'next_lunar_quarters',
 'next_solar_stations',
 'path_in_sky',
 'previous_lunar_quarters',
 'previous_solar_stations',
 'when_appear_disappear',
 'when_rise_culminates_set',
 'where_in_sky']